In [1]:
!pip install transformers[torch] -q
!pip install datasets -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.4 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/My Drive/HSE/programming/NLP/project'
! ls

/content/drive/My Drive/HSE/programming/NLP/project
 bert_model.pth			  project.ipynb       train_cats.txt
 dev_pred_aspects.txt		  test_aspects.txt    train_reviews.txt
 entities_result.json		  TEST_BIO.csv	      train_split_aspects.txt
'entities_result_test (1).json'   test_cats.txt       train_split_cats.txt
 entities_result_test.json	  test_reviews.txt    train_split_reviews.txt
 project1.ipynb			  train_aspects.txt   updated_dev_pred_aspects.txt


In [3]:
import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertTokenizerFast, BertForSequenceClassification, BertForTokenClassification
from torch.utils.data import DataLoader, Dataset
import torch
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [4]:
# Загрузка данных
df_aspects = pd.read_csv('/content/drive/MyDrive/NLP Course/Data/train/train_split_aspects (2).txt', sep='\t', names=['review_id', 'aspect_category', 'aspect_text', 'start_pos', 'end_pos', 'sentiment'])
df_reviews = pd.read_csv('/content/drive/MyDrive/NLP Course/Data/train/train_split_reviews.txt', sep='\t', names=['review_id', 'review_text'])

# Объединение данных
merged_data = pd.merge(df_aspects, df_reviews, on='review_id')

In [5]:
merged_data.head()

,review_id,aspect_category,aspect_text,start_pos,end_pos,sentiment,review_text
0,3976,Whole,ресторане,71,80,neutral,"День 8-го марта прошёл, можно и итоги подвести..."
1,3976,Whole,ресторанах,198,208,neutral,"День 8-го марта прошёл, можно и итоги подвести..."
2,3976,Whole,ресторане,256,265,neutral,"День 8-го марта прошёл, можно и итоги подвести..."
3,3976,Service,Столик бронировали,267,285,neutral,"День 8-го марта прошёл, можно и итоги подвести..."
4,3976,Service,администратор,322,335,positive,"День 8-го марта прошёл, можно и итоги подвести..."


In [6]:
# Функция для извлечения контекста вокруг аспекта с учетом целых слов
def extract_context_with_category(text, start, end, category, window=50):
    # Находим начало контекста и пробел назад от начальной позиции аспекта
    start_idx = max(0, start - window)
    while start_idx > 0 and text[start_idx] != ' ':
        start_idx -= 1

    # Находим конец контекста и пробел вперед от конечной позиции аспекта
    end_idx = min(len(text), end + window)
    while end_idx < len(text) and text[end_idx] != ' ':
        end_idx += 1

    context = text[start_idx:end_idx].strip()
    return category + " " + context  # Добавление категории аспекта

In [ ]:
merged_data['context_aspect'] = merged_data.apply(
    lambda row: extract_context_with_category(row['review_text'], row['start_pos'], row['end_pos'], row['aspect_category']), axis=1
)

In [ ]:
merged_data.head(10)

,review_id,aspect_category,aspect_text,start_pos,end_pos,sentiment,review_text,context_aspect
0,3976,Whole,ресторане,71,80,neutral,"День 8-го марта прошёл, можно и итоги подвести...","Whole прошёл, можно и итоги подвести. Решил на..."
1,3976,Whole,ресторанах,198,208,neutral,"День 8-го марта прошёл, можно и итоги подвести...",Whole edik077 и Rules77777и понял что либо мы ...
2,3976,Whole,ресторане,256,265,neutral,"День 8-го марта прошёл, можно и итоги подвести...","Whole ресторанах, либо у ребят что-то незалади..."
3,3976,Service,Столик бронировали,267,285,neutral,"День 8-го марта прошёл, можно и итоги подвести...",Service ребят что-то незаладилось. Но теперь о...
4,3976,Service,администратор,322,335,positive,"День 8-го марта прошёл, можно и итоги подвести...",Service Столик бронировали заранее и сделали т...
5,3976,Service,предварительный заказ,349,370,positive,"День 8-го марта прошёл, можно и итоги подвести...",Service сделали так как предложил администрато...
6,3976,Whole,ресторан,413,421,neutral,"День 8-го марта прошёл, можно и итоги подвести...","Whole предварительный заказ, когда придя увиде..."
7,3976,Whole,ресторане,476,485,neutral,"День 8-го марта прошёл, можно и итоги подвести...",Whole поняли что совет нам дали действительно ...
8,3976,Food,горячее блюдо,524,537,neutral,"День 8-го марта прошёл, можно и итоги подвести...","Food в ресторане было человек 70-80, тут дейст..."
9,3976,Food,Меню,564,568,positive,"День 8-го марта прошёл, можно и итоги подвести...",Food действительно горячее блюдо можно ждать в...


In [ ]:
# Создание обучающего набора данных
training_data_context = merged_data[['context_aspect', 'sentiment']].copy()
label_dict = {'negative': 0, 'neutral': 1, 'positive': 2, 'both': 3}

# Преобразование меток в числовые значения
training_data_context.loc[:, 'sentiment_label'] = training_data_context['sentiment'].replace(label_dict)

train_texts, val_texts, train_labels, val_labels = train_test_split(training_data_context['context_aspect'], training_data_context['sentiment_label'], test_size=0.1)

In [ ]:
training_data_context.dtypes

context_aspect     object
sentiment          object
sentiment_label     int64
dtype: object

In [ ]:
model_name = 'DeepPavlov/rubert-base-cased'

# Загрузка токенизатора
tokenizer = BertTokenizer.from_pretrained(model_name, model_max_length=512)

# Функция для токенизации
def tokenize_and_format(texts, labels):
    input_ids = []
    attention_masks = []

    for text in texts:
        encoded = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=64,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    return input_ids, attention_masks

tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

In [ ]:
train_labels_list = train_labels.replace(label_dict).tolist()
val_labels_list = val_labels.replace(label_dict).tolist()

train_inputs, train_masks = tokenize_and_format(train_texts, train_labels_list)
val_inputs, val_masks = tokenize_and_format(val_texts, val_labels_list)

In [ ]:
train_labels_tensor = torch.tensor(train_labels_list)
val_labels_tensor = torch.tensor(val_labels_list)

class SentimentDataset(Dataset):
    def __init__(self, inputs, masks, labels):
        self.inputs = inputs
        self.masks = masks
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {
            'input_ids': self.inputs[idx],
            'attention_mask': self.masks[idx],
            'labels': self.labels[idx]
        }
        return item

train_dataset = SentimentDataset(train_inputs, train_masks, train_labels_tensor)
val_dataset = SentimentDataset(val_inputs, val_masks, val_labels_tensor)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32)

In [ ]:
# Загрузка предварительно обученной модели BERT
model = BertForSequenceClassification.from_pretrained('DeepPavlov/rubert-base-cased', num_labels=len(label_dict))

optimizer = AdamW(model.parameters(), lr=2e-5)

device = "cuda" if torch.cuda.is_available() else "cpu"
epochs = 4

model.to(device)

for epoch in range(epochs):
    model.train()
    for batch in tqdm(train_dataloader):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch} completed")

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 100/100 [00:35<00:00,  2.82it/s]


Epoch 0 completed


100%|██████████| 100/100 [00:35<00:00,  2.78it/s]


Epoch 1 completed


100%|██████████| 100/100 [00:38<00:00,  2.63it/s]


Epoch 2 completed


100%|██████████| 100/100 [00:37<00:00,  2.66it/s]

Epoch 3 completed


In [ ]:
model.eval()

total_eval_accuracy = 0
total_eval_loss = 0

with torch.no_grad():
    for batch in val_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)

        loss = outputs[0]
        total_eval_loss += loss.item()

        logits = outputs[1]
        predictions = torch.argmax(logits, dim=1).flatten()
        accuracy = (predictions == labels).cpu().numpy().mean() * 100
        total_eval_accuracy += accuracy

avg_val_accuracy = total_eval_accuracy / len(val_dataloader)
avg_val_loss = total_eval_loss / len(val_dataloader)

print(f"Точность на валидационном наборе: {avg_val_accuracy}%")
print(f"Средняя потеря на валидационном наборе: {avg_val_loss}")

Точность на валидационном наборе: 80.72916666666667%
Средняя потеря на валидационном наборе: 0.6293261917307973


In [ ]:
def predict_sentiment(text, model, tokenizer):
    encoded_review = tokenizer.encode_plus(
        text,
        max_length=64,
        add_special_tokens=True,
        return_token_type_ids=False,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt',
        truncation=True
    )

    input_ids = encoded_review['input_ids'].to(device)
    attention_mask = encoded_review['attention_mask'].to(device)

    with torch.no_grad():
        output = model(input_ids, attention_mask=attention_mask)
        logits = output[0]

    probabilities = torch.softmax(logits, dim=1).flatten()
    print(probabilities)

    # Возвращаем класс с наибольшей вероятностью
    return torch.argmax(probabilities).item()

text = "в восторге. Прекрасный интерьер, хорошее обслуживание, быстро, ненавязчиво. "
predicted_class = predict_sentiment(text, model, tokenizer)
print(f"Предсказанная тональность: {list(label_dict.keys())[list(label_dict.values()).index(predicted_class)]}")

tensor([0.0011, 0.0025, 0.9945, 0.0019], device='cuda:0')
Предсказанная тональность: positive


In [ ]:
predict_sentiment("Испортилось все! Официанты работают ужасно, не дозовешься", model, tokenizer)

tensor([0.9063, 0.0104, 0.0638, 0.0195], device='cuda:0')


0

In [ ]:
# Сохранение модели
model_save_path = "bert_model.pth"
torch.save(model.state_dict(), model_save_path)

In [ ]:
label_dict = {'negative': 0, 'neutral': 1, 'positive': 2, 'both': 3}
device = "cuda" if torch.cuda.is_available() else "cpu"
model_path = "bert_model.pth"
model_name = 'DeepPavlov/rubert-base-cased'

model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(label_dict))

# Загрузка сохраненных весов
model.load_state_dict(torch.load(model_path))

model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [ ]:
dev_aspects = pd.read_csv('dev_pred_aspects.txt', sep='\t', names=['review_id', 'aspect_category', 'aspect_text', 'start_pos', 'end_pos', 'sentiment'])
dev_reviews = pd.read_csv('test_reviews.txt', sep='\t', names=['review_id', 'review_text'])

# Объединение данных
merged_data = pd.merge(dev_aspects, dev_reviews, on='review_id')

# Добавление контекста к данным
merged_data['context'] = merged_data.apply(lambda row: extract_context_with_category(row['review_text'], row['start_pos'], row['end_pos'], row['aspect_category']), axis=1)

In [ ]:
tokenizer = BertTokenizer.from_pretrained(model_name, model_max_length=512)

def predict_sentiment(text):
    encoded_input = tokenizer.encode_plus(
        text,
        max_length=64,
        add_special_tokens=True,
        return_attention_mask=True,
        padding='max_length',
        return_tensors='pt',
        truncation=True
    )
    input_ids = encoded_input['input_ids'].to(device)
    attention_mask = encoded_input['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        prediction = torch.argmax(outputs.logits, dim=1).item()

    # Возвращаем тональность аспекта
    return list(label_dict.keys())[list(label_dict.values()).index(prediction)]

# Применение модели к каждому контексту и обновление значения тональности
dev_aspects['sentiment'] = merged_data['context'].apply(predict_sentiment)

dev_aspects.to_csv('updated_dev_pred_aspects.txt', sep='\t', index=False)

In [ ]:
# Загрузка предсказанных и истинных данных
df_predicted = pd.read_csv('updated_dev_pred_aspects.txt', sep='\t')
df_true = pd.read_csv('test_aspects.txt', sep='\t', header=None, names=['review_id', 'aspect_category', 'aspect_text', 'start_pos', 'end_pos', 'sentiment'])

In [ ]:
# Объединение данных по ключевым столбцам
merged_data = pd.merge(df_predicted, df_true, on=['review_id', 'aspect_text', 'start_pos', 'end_pos'], suffixes=('_pred', '_true'))

# Расчет доли верных ответов
accuracy = (merged_data['sentiment_pred'] == merged_data['sentiment_true']).mean()
print(f"Точность: {accuracy}")

Точность: 0.8016431924882629


In [ ]:
merged_data.head(10)

,review_id,aspect_category_pred,aspect_text,start_pos,end_pos,sentiment_pred,aspect_category_true,sentiment_true
0,38835,Whole,заведении,51,60,neutral,Whole,neutral
1,38835,Service,сервисом,98,106,positive,Service,positive
2,38835,Service,Встретила,108,117,positive,Service,positive
3,38835,Service,девушка,136,143,positive,Service,positive
4,38835,Service,персонал,179,187,positive,Service,positive
5,38835,Service,официант,230,238,positive,Service,positive
6,38835,Food,салата,363,369,neutral,Food,neutral
7,38835,Food,свинину,415,422,neutral,Food,neutral
8,38835,Food,кальян,424,430,neutral,Food,neutral
9,38835,Food,десерт,452,458,neutral,Food,neutral


In [ ]:
merged_data[merged_data['sentiment_pred'] != merged_data['sentiment_true']].head(10)

,review_id,aspect_category_pred,aspect_text,start_pos,end_pos,sentiment_pred,aspect_category_true,sentiment_true
10,38835,Food,Утка,488,492,neutral,Food,negative
12,38835,Whole,ресторанах,677,687,negative,Whole,positive
15,38835,Food,порции,779,785,positive,Food,negative
16,38835,Whole,впечатление,836,847,positive,Whole,both
19,1368,Whole,пабе,24,28,neutral,Whole,positive
24,1368,Service,официант,395,403,positive,Service,neutral
25,1368,Service,принес меню,404,415,positive,Service,neutral
28,1368,Food,блюд,493,497,neutral,Food,positive
29,1368,Food,меню,542,546,neutral,Food,positive
41,1368,Service,сдачу ждали,1046,1057,neutral,Service,negative


In [ ]:
merged_data[merged_data['aspect_category_pred'] != merged_data['aspect_category_true']].head(10)

,review_id,aspect_category_pred,aspect_text,start_pos,end_pos,sentiment_pred,aspect_category_true,sentiment_true
45,6668,Service,Накормили,393,402,positive,Food,positive
133,28083,Service,горячее,832,839,negative,Food,neutral
152,3152,Food,тамаду,599,605,neutral,Service,neutral
300,3906,Service,музыкальное сопровождение,163,188,positive,Interior,positive
306,3906,Whole,Постное меню,480,492,positive,Food,both
311,32442,Interior,места для танцев,180,196,positive,Service,positive
341,29298,Service,охраны,144,150,negative,Interior,neutral
404,20021,Price,барная карта,512,524,positive,Food,positive
405,20021,Price,Выбор еды,607,616,negative,Food,negative
467,32859,Interior,стол,740,744,positive,Service,positive


In [ ]:
(merged_data['aspect_category_pred'] != merged_data['aspect_category_true']).count()

852

## Добавление синтаксического окна контекста для аспекта

In [7]:
! python -m spacy download ru_core_news_sm

✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [8]:
import spacy

# Загрузка модели spacy для языка, на котором написан текст
nlp = spacy.load("ru_core_news_sm")  # Или другая модель

def syntactic_context(aspect_tokens, k):
    L = set(aspect_tokens)
    current_tokens = aspect_tokens
    while k > 0:
        new_tokens = set()
        for token in current_tokens:
            # Добавление родительского узла
            if token.head not in L:
                new_tokens.add(token.head)
            # Добавление дочерних узлов
            new_tokens.update([child for child in token.children if child not in L])
        L.update(new_tokens)
        current_tokens = new_tokens
        k -= 1
    # Возвращение отсортированной строки с текстом токенов
    return ' '.join([token.text for token in sorted(L, key=lambda x: x.i)])

def extract_syntactical_window(text, start_char, end_char, category, k):
    doc = nlp(text)
    # Находим все токены, попадающие в диапазон аспекта
    aspect_tokens = [token for token in doc if not (token.idx >= end_char or token.idx + len(token.text) <= start_char)]
    # Применяем алгоритм синтаксического окна
    context = syntactic_context(aspect_tokens, k)
    return category + " " + context

In [ ]:
# Пример использования
text_example = "Отмечали в этом ресторане день рождение на первом этаже в субботу вечером. Хочу выразить большую благодарность прежде всего руководству ресторана, обслуживающему персоналу и конечно же тем сотрудникам, которые готовят посетителям заведения столь вкусные блюда. Понравилось абсолютно все. Конечно же впечатляет интерьер: уютно и красиво. Стол был шикарный. Нас обслуживал официант Бахрам, спасибо ему. Хочу отметить, что для ресторана такого уровня цены не высокие."
aspect_example_start = text_example.find('официант')
aspect_example_end = aspect_example_start + len('официант')
context = extract_syntactical_window(text_example, aspect_example_start, aspect_example_end, k=3)
print(context)

In [ ]:
text_example = "Всем доброго дня! Зашел посмотреть телефон заведения, чтобы заказать столик, но решил написать и отзыв ))) Ходим сюда часто, и с друзьями, и с семьёй. Здесь даже отмечал мой друг день рождения, было весело и вкусно. Последний раз посещали ресторан в субботу 26 числа. Пришли с другом и с женами часов в 5 вечера, народу было на удивление немного. Мы, мужчины, пили пиво, наши женщины пили красное вино, довольно неплохое. Ели роллы (традиционную филадельфию и острые запеченные), друзья брали супы, сказали, что вкусные. На горячее я брал перечный стейк (люблю его очень). Не всегда угадывают с прожаркой, но в этот раз был идеальный. Посидели недолго, т.к. торопились в кино. Надеюсь, в эти выходные посидим подольше и дождемся живой музыки. В целом очень достойное место, особенно за такие деньги."
aspect_example_start = text_example.find('перечный стейк')
aspect_example_end = aspect_example_start + len('перечный стейк')
context = extract_syntactical_window(text_example, aspect_example_start, aspect_example_end, k=3)
print(context)

In [12]:
from tqdm import tqdm
tqdm.pandas()

In [13]:
merged_data['syntactical_context'] = merged_data.progress_apply(
    lambda row: extract_syntactical_window(row['review_text'], row['start_pos'], row['end_pos'], row['aspect_category'], k=3), axis=1
)

100%|██████████| 3553/3553 [05:02<00:00, 11.76it/s]


In [15]:
merged_data

,review_id,aspect_category,aspect_text,start_pos,end_pos,sentiment,review_text,syntactical_context
0,3976,Whole,ресторане,71,80,neutral,"День 8-го марта прошёл, можно и итоги подвести...",Whole Решил написать отзыв о ресторане в котором
1,3976,Whole,ресторанах,198,208,neutral,"День 8-го марта прошёл, можно и итоги подвести...",Whole Решил понял что либо мы были вразных рес...
2,3976,Whole,ресторане,256,265,neutral,"День 8-го марта прошёл, можно и итоги подвести...",Whole Но теперь о ресторане .
3,3976,Service,Столик бронировали,267,285,neutral,"День 8-го марта прошёл, можно и итоги подвести...",Service Столик бронировали заранее и сделали т...
4,3976,Service,администратор,322,335,positive,"День 8-го марта прошёл, можно и итоги подвести...",Service бронировали и сделали так как предложи...
...,...,...,...,...,...,...,...,...
3548,16630,Service,обслуживание,85,97,positive,Уютная и тёплая домашняя обстановка! Милый и о...,"Service Милый отзывчивый персонал , очень вним..."
3549,16630,Food,Еда,99,102,positive,Уютная и тёплая домашняя обстановка! Милый и о...,"Food Еда , стоит ли говорить трудно ."
3550,16630,Service,персоналу,244,253,positive,Уютная и тёплая домашняя обстановка! Милый и о...,"Service Спасибо персоналу , ему удается создат..."
3551,16630,Whole,ресторан,294,302,positive,Уютная и тёплая домашняя обстановка! Милый и о...,"Whole ощущение , что этот ресторан - особенное..."


In [16]:
# Создание обучающего набора данных
training_data_context = merged_data[['syntactical_context', 'sentiment']].copy()
label_dict = {'negative': 0, 'neutral': 1, 'positive': 2, 'both': 3}

# Преобразование меток в числовые значения
training_data_context.loc[:, 'sentiment_label'] = training_data_context['sentiment'].replace(label_dict)

train_texts, val_texts, train_labels, val_labels = train_test_split(training_data_context['syntactical_context'], training_data_context['sentiment_label'], test_size=0.1)

In [17]:
model_name = 'DeepPavlov/rubert-base-cased'

# Загрузка токенизатора
tokenizer = BertTokenizer.from_pretrained(model_name, model_max_length=512)

# Функция для токенизации
def tokenize_and_format(texts, labels):
    input_ids = []
    attention_masks = []

    for text in texts:
        encoded = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=64,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    return input_ids, attention_masks

tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

In [18]:
train_labels_list = train_labels.replace(label_dict).tolist()
val_labels_list = val_labels.replace(label_dict).tolist()

train_inputs, train_masks = tokenize_and_format(train_texts, train_labels_list)
val_inputs, val_masks = tokenize_and_format(val_texts, val_labels_list)

In [19]:
train_labels_tensor = torch.tensor(train_labels_list)
val_labels_tensor = torch.tensor(val_labels_list)

class SentimentDataset(Dataset):
    def __init__(self, inputs, masks, labels):
        self.inputs = inputs
        self.masks = masks
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {
            'input_ids': self.inputs[idx],
            'attention_mask': self.masks[idx],
            'labels': self.labels[idx]
        }
        return item

train_dataset = SentimentDataset(train_inputs, train_masks, train_labels_tensor)
val_dataset = SentimentDataset(val_inputs, val_masks, val_labels_tensor)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32)

In [20]:
# Загрузка предварительно обученной модели BERT
model = BertForSequenceClassification.from_pretrained('DeepPavlov/rubert-base-cased', num_labels=len(label_dict))

optimizer = AdamW(model.parameters(), lr=2e-5)

device = "cuda" if torch.cuda.is_available() else "cpu"
epochs = 4

model.to(device)

for epoch in range(epochs):
    model.train()
    for batch in tqdm(train_dataloader):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch} completed")

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 100/100 [00:33<00:00,  3.02it/s]


Epoch 0 completed


100%|██████████| 100/100 [00:33<00:00,  2.98it/s]


Epoch 1 completed


100%|██████████| 100/100 [00:34<00:00,  2.87it/s]


Epoch 2 completed


100%|██████████| 100/100 [00:36<00:00,  2.75it/s]

Epoch 3 completed


In [21]:
model.eval()

total_eval_accuracy = 0
total_eval_loss = 0

with torch.no_grad():
    for batch in val_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)

        loss = outputs[0]
        total_eval_loss += loss.item()

        logits = outputs[1]
        predictions = torch.argmax(logits, dim=1).flatten()
        accuracy = (predictions == labels).cpu().numpy().mean() * 100
        total_eval_accuracy += accuracy

avg_val_accuracy = total_eval_accuracy / len(val_dataloader)
avg_val_loss = total_eval_loss / len(val_dataloader)

print(f"Точность на валидационном наборе: {avg_val_accuracy}%")
print(f"Средняя потеря на валидационном наборе: {avg_val_loss}")

Точность на валидационном наборе: 76.5625%
Средняя потеря на валидационном наборе: 0.7453712330510219


In [22]:
# Сохранение модели
model_save_path = "bert_model_with_syntax.pth"
torch.save(model.state_dict(), model_save_path)

In [23]:
label_dict = {'negative': 0, 'neutral': 1, 'positive': 2, 'both': 3}
device = "cuda" if torch.cuda.is_available() else "cpu"
model_path = "bert_model_with_syntax.pth"
model_name = 'DeepPavlov/rubert-base-cased'

model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(label_dict))

# Загрузка сохраненных весов
model.load_state_dict(torch.load(model_path))

model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [26]:
dev_aspects = pd.read_csv('/content/drive/MyDrive/NLP Course/dev_pred_aspects.txt', sep='\t', names=['review_id', 'aspect_category', 'aspect_text', 'start_pos', 'end_pos', 'sentiment'])
dev_reviews = pd.read_csv('/content/drive/MyDrive/NLP Course/Data/test/test_reviews.txt', sep='\t', names=['review_id', 'review_text'])

# Объединение данных
merged_data = pd.merge(dev_aspects, dev_reviews, on='review_id')

# Добавление контекста к данным
merged_data['context'] = merged_data.apply(lambda row: extract_context_with_category(row['review_text'], row['start_pos'], row['end_pos'], row['aspect_category']), axis=1)

In [27]:
tokenizer = BertTokenizer.from_pretrained(model_name, model_max_length=512)

def predict_sentiment(text):
    encoded_input = tokenizer.encode_plus(
        text,
        max_length=64,
        add_special_tokens=True,
        return_attention_mask=True,
        padding='max_length',
        return_tensors='pt',
        truncation=True
    )
    input_ids = encoded_input['input_ids'].to(device)
    attention_mask = encoded_input['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        prediction = torch.argmax(outputs.logits, dim=1).item()

    # Возвращаем тональность аспекта
    return list(label_dict.keys())[list(label_dict.values()).index(prediction)]

# Применение модели к каждому контексту и обновление значения тональности
dev_aspects['sentiment'] = merged_data['context'].apply(predict_sentiment)

dev_aspects.to_csv('dev_pred_aspects_syntax.txt', sep='\t', index=False)

In [28]:
# Загрузка предсказанных и истинных данных
df_predicted = pd.read_csv('dev_pred_aspects_syntax.txt', sep='\t')
df_true = pd.read_csv('/content/drive/MyDrive/NLP Course/Data/test/test_aspects.txt', sep='\t', header=None, names=['review_id', 'aspect_category', 'aspect_text', 'start_pos', 'end_pos', 'sentiment'])

In [29]:
# Объединение данных по ключевым столбцам
merged_data = pd.merge(df_predicted, df_true, on=['review_id', 'aspect_text', 'start_pos', 'end_pos'], suffixes=('_pred', '_true'))

# Расчет доли верных ответов
accuracy = (merged_data['sentiment_pred'] == merged_data['sentiment_true']).mean()
print(f"Точность: {accuracy}")

Точность: 0.7727891156462585
